# Making publication quality elemental maps

In this notebook we show how data EELS (or EDS) data processed with `HyperSpy` and `eXSpy` can be made into high quality figures and plots. We utilize the python library `matplotlib`, which we use to make customized figures for use in presentations or journal publications.

The Notebook example here utilizes EELS data processed using the `EELS_elemental_mapping` Jupyter Notebook in https://github.com/hyperspy/exspy-demos/tree/main/EELS. We'll be making elemental maps from the intensity of the model of the Cu-L$_3$ and Zn-L$_3$ components. The HAADF image will also be plotted, for comparison with the elemental distribution.

For more information about the material system and how the results of this processing can be used, see the paper: https://doi.org/10.1016/j.cattod.2019.02.045.

While this notebook focuses on plotting EELS data, it can equally well be used for plotting elemental maps from EDS data.

---------

* 2017/09/27: Initial version by Ida Hjorth
* 2024/4/1: Update to work with HyperSpy 2.0, by Magnus Nord

## Importing the modules and loading the data

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import hyperspy.api as hs

Loading the maps

In [ ]:
Cu_map = hs.load('datasets/Cu_map.hspy')
Zn_map = hs.load('datasets/Zn_map.hspy')

Extracting the intensity of each element from the model.

**Note:** as the components L$_3$, L$_2$ and L$_1$ are connected during the fitting procedure, we only need to visualize one of them to get a relative elemental map.

In [ ]:
s_haadf = hs.load('datasets/CuZn_HAADF.hspy')

`GridSpec` is a convenient function to use when creating several subplots. We create axes for the elemental maps, HAADF signal as well as the colorbars that will be used indicate the numerical intensity.

For more information on how to use `GridSpec`, you can access the docstring by running `gridspec.GridSpec?` in a cell.  (Note the question mark)

In [ ]:
fig = plt.figure(figsize=(5.5, 2.7))
gs = gridspec.GridSpec(30, 3)
ax_cu = fig.add_subplot(gs[0:-2, 0])
ax_zn = fig.add_subplot(gs[0:-2, 1])
ax_haadf = fig.add_subplot(gs[0:-2, 2])
cbar_cu =  fig.add_subplot(gs[-1, 0]) # Colorbars are much thinner than the map axes (1/30 of the height of the image)
cbar_zn =  fig.add_subplot(gs[-1, 1])
cbar_haadf =  fig.add_subplot(gs[-1, 2])

The signals are plotted using `imshow`.

In [ ]:
cax_cu = ax_cu.imshow(
    Cu_map.data,
    interpolation='nearest',
    extent=Cu_map.axes_manager.signal_extent,
)

In [ ]:
cax_zn = ax_zn.imshow(
    Zn_map.data,
    interpolation='nearest',
    extent=Zn_map.axes_manager.signal_extent,
)

NumPy functions, such as `flipud`, `fliplr` and `rot90` can also be used to give the image the correct orientation. Here,`flipud` is used to give the HAADF image the same orientation as the elemental maps.

In [ ]:
ax_haadf.imshow(
    s_haadf.data,
    interpolation='nearest',
    extent=s_haadf.axes_manager.signal_extent,
)

Disable axis ticks

In [ ]:
ax_haadf.set_xticks([])
ax_cu.set_xticks([])
ax_zn.set_xticks([])
ax_zn.set_yticks([])
ax_cu.set_yticks([])
ax_haadf.set_yticks([])

Scalebars can be added by using `AnchoredSizeBar`. 

In [ ]:
fontprops = fm.FontProperties(size=18)
scalebar0 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 2, # length of bar, label, loc
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar1 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 2,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar2 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 2,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
ax_cu.add_artist(scalebar0)
ax_zn.add_artist(scalebar1)
ax_haadf.add_artist(scalebar2)

We add labels to indicate what is shown in each image.

In [ ]:
ax_cu.text(0.05, 0.05, 'Cu', color='white', size=18, transform=ax_cu.transAxes)
ax_zn.text(0.05, 0.05, 'Zn', color='white', size=18, transform=ax_zn.transAxes)
ax_haadf.text(0.05, 0.05, 'HAADF', color='white', size=18, transform=ax_haadf.transAxes)

Then colorbars are added. 

In [ ]:
cb_zn = fig.colorbar(ax_zn.images[0], cax=cbar_zn, extend='both', orientation='horizontal', label="Relative Zn, [a.u.]")
cb_cu = fig.colorbar(ax_cu.images[0], cax=cbar_cu,extend='both', orientation='horizontal', label="Relative Cu, [a.u.]")
cb_haadf = fig.colorbar(ax_haadf.images[0], cax=cbar_haadf, extend='both', orientation='horizontal', label="HAADF, [a.u.]")

In [ ]:
cb_cu.set_ticks([0, 5, 10, 15])
cb_zn.set_ticks([0, 5, 10, 15, 20, 25])
cb_haadf.set_ticks([0, 1000, 2000])

The color schemes `viridis`, `inferno`, `plasma` and `magma` are nice to use as they are grayscale compatible, perceptually uniform and colorblind-proof. For more info on these color maps, see https://www.youtube.com/watch?v=xAoljeRJ3lU

In [ ]:
cax_zn.set_clim(vmin=0, vmax=25)
cax_zn.set_cmap('inferno')
cax_cu.set_clim(vmin=0, vmax=15)
cax_cu.set_cmap('viridis')

In [ ]:
fig.subplots_adjust(left=0, bottom=0.15, right=1, top=0.98)

Saving the matplotlib figure object as a png-file. Here, the resolution (via dots per inch, `dpi`) is set to a high value: 300. 

In [ ]:
fig.savefig("elemental_map.png", dpi=300)

We can see the end result inside the Notebook by using `plt.show()`

In [ ]:
plt.show()